<h2><center> MACHINE LEARNING PROJECT - N. 6 </center></h2>

<h3><center> Subscript nr. 1 - Import procedure </center></h3>

This notebook contains preprocessing code, which is relevant for the creation of the dataframe used in analysis.

### Setting path

In [1]:
%run libraries.ipynb

In [2]:
delimiter = "\\" # for windows
#delimiter = "/"  # for os system

path_raw = ".." + delimiter + 'results'

### Loading covid and geographical data

In [3]:
covid_path = ".." + delimiter + 'results' + delimiter + "covid.xlsx"
covid_info = pd.read_excel(covid_path)

covid_cols = list(covid_info.columns)
covid_info = covid_info.rename( columns = {
                covid_cols[-3]: 'Collect',
                covid_cols[-2]: 'Death',
                covid_cols[-1]: 'Cases' } )

geo_path = ".." + delimiter + 'results' + delimiter + "geo_info.csv"
geo_info = pd.read_csv(geo_path)
geo_info = geo_info.rename(columns = {"city":'City'})

### Lists initialization

Each element of lists is the mean of the variable for a city in a range from twenty to thirty days before first death.

In [4]:
listmean_t2m = []
listmean_rh = []
listmean_ah = []
listmean_q = []

Creation of two lists containing respectively cities and countries names.

In [5]:
city_list = np.array(covid_info["City"]).tolist()
country_list = np.array(covid_info["Country"]).tolist()

### Storing means of weather variables

In [6]:
%run new_variable_function.ipynb

Loop that allows to cut the data by time periods and adding means to its list.

In [7]:
for city in city_list:
    df = pd.read_csv(path_raw + delimiter + city + ".csv")
    
    index_collect = covid_info.index[covid_info["City"] == city].to_list()
    collect_date = covid_info["Collect"][index_collect] # extract date of data collection to subset
    
    start_date = collect_date - datetime.timedelta(days = 30)
    start_date = start_date.astype(str).to_list()[0]
    end_date = collect_date - datetime.timedelta(days = 20)
    end_date = end_date.astype(str).to_list()[0]
    
    df[["Date","Hour"]] = df.date.str.split(" ", expand = True) # True is to return dataframe
    df["Date"] = df["Date"].astype(str)
    
    index_start = df.index[df["Date"] == start_date].to_list()[0] # consider 00:00
    index_end = df.index[df["Date"] == end_date].to_list()[-1] # consider 23:00
    
    df = df[(index_start+1):(index_end+1)] # subset dataframe
    df = add_vars(df)
    
    listmean_t2m.append(np.mean(df["t2m"]))
    listmean_rh.append(np.mean(df["RH"]))
    listmean_ah.append(np.mean(df["AH"]))
    listmean_q.append(np.mean(df["Q"]))

### Final dataset

Creation of a dataframe, through a temporary dictionary for each row.

In [8]:
list_dicts = []

for i in range(len(city_list)):
    row_temp_dict = { 'City' : city_list[i],
                      'Country' : country_list[i],
                      'Latitude' : geo_info.iloc[i]['lat'],
                      'TempCels' : round( listmean_t2m[i] , 5 ) - 273.15, # Convert Kelvin to Celsius
                      'SpecHum' : round( listmean_q[i], 5),
                      'RelHum' : round( listmean_rh[i] , 5 ),
                      'AbsHum' : round( listmean_ah[i] , 5) }
    
    list_dicts.append(row_temp_dict)


df_converted = pd.DataFrame.from_dict(list_dicts, orient = 'columns') # keys of dicts are cols

The dataset containing weather variables and the one containing Covid-19 cases and deaths are merged. The resulting dataset is stored in the <i>results</i> folder.

In [9]:
new_df = pd.merge(df_converted, covid_info, on = ["City","Country"], how = 'inner')
new_df.to_excel(path_raw + delimiter + "final_dataset.xlsx", index = False)